# mRS distribution derivation

k

In [60]:
import numpy as np

In [61]:
def fudge_sum_one(dist, dp=3):
    """
    Force sum of a distribution to be exactly 1.
    
    Nudge the smallest fractions down or the largest fractions up as required.
    """
    if np.round(np.sum(np.round(dist, dp)), dp) == 1.0:
        # Nothing to see here.
        return np.round(dist, dp)

    # Add or subtract from the mRS proportions until the sum is 1.
    # Start by adding to numbers with large fractional parts
    # or subtracting from numbers with small fractional parts.

    
    # Store the integer part of each value,
    # the fractional part of each value,
    # and values to track how many times the integer part
    # has been fudged upwards and downwards.
    # Final digit of each value:
    success = False
    while success == False:
        # Convert to integers.
        dist = dist * 10**dp
        dist_int = dist.astype(int)
        dist_frac = dist % dist.astype(int)
        target = 10 ** dp
        # final_digits = np.round((10 * ((dist * 10**(dp - 1)) % 1)), 0).astype(int)
        # if np.all(final_digits == 0.0):
        if np.all(dist_frac == 0.0):
            # If the dist is already rounded to the requested dp,
            # try the next digit up.
            dp -= 1
        else:
            # Use this precision.
            success = True
            
    arr = np.zeros((len(dist), 4), dtype=int)
    arr[:, 0] = dist_int
    arr[:, 1] = (dist_frac * 1000).astype(int)
    
    # Cut off this process after 20 loops.
    loops = 0
    sum_dist = np.sum(arr[:, 0])

    while loops < 20:
        if sum_dist < target:
            # Pick out the values that have been added to
            # the fewest times.
            min_change = np.min(arr[:, 2])
            inds_min_change = np.where(arr[:, 2] == min_change)
            # Of these, pick out the value with the largest
            # fractional part.
            largest_frac = np.max(arr[inds_min_change, 1])
            ind_largest_frac = np.where(
                (arr[:, 2] == min_change) &
                (arr[:, 1] == largest_frac)
            )
            if len(ind_largest_frac[0]) > 1:
                # Arbitrarily pick the lowest mRS if multiple options.
                ind_largest_frac = ind_largest_frac[0][0]
            # Add one to the final digit of this mRS proportion
            # and record the change in column 2.
            arr[ind_largest_frac, 0] += 1
            # arr[ind_largest_frac, 1] +=  
            arr[ind_largest_frac, 2] += 1
        elif sum_dist > target:
            # Pick out the values that have been subtracted from
            # the fewest times.
            min_change = np.min(arr[:, 3])
            inds_min_change = np.where(arr[:, 3] == min_change)
            # Of these, pick out the value with the smallest
            # fractional part.
            smallest_frac = np.min(arr[inds_min_change, 1])
            ind_smallest_frac = np.where(
                (arr[:, 3] == min_change) &
                (arr[:, 1] == smallest_frac)
            )
            if len(ind_smallest_frac[0]) > 1:
                # Arbitrarily pick the lowest mRS if multiple options.
                ind_smallest_frac = ind_smallest_frac[0][0]
            # Subtract one from the final digit of this mRS proportion
            # and record the change in column 3.
            arr[ind_smallest_frac, 0] -= 1
            # arr[ind_smallest_frac, 1] -= 1
            arr[ind_smallest_frac, 3] += 1
        sum_dist = np.sum(arr[:, 0])
        if sum_dist == target:
            loops = 20
        else:
            loops += 1

    # Take the new fudged distribution:
    dist_fudged = arr[:, 0] / 10**dp

    return dist_fudged

Store the derived mRS distributions in this dictionary:

In [62]:
mrs_dists = dict()

## Pre-stroke data

![](images/data_sources_pre-stroke.png)

In [63]:
pre_stroke_nlvo = np.array(
    [0.582771, 0.163513, 0.103988, 0.101109, 0.041796, 0.006822, 0.0000])
pre_stroke_lvo = np.array(
    [0.407796, 0.143538, 0.120133, 0.166050, 0.118023, 0.044460, 0.0000])

Round to 3 decimal places and force sum to be exactly 1 by nudging the smallest fractions down or the largest fractions up as required.

In [64]:
# Use append to make sure the mRS=6 value can't be changed.
pre_stroke_nlvo = np.append(fudge_sum_one(pre_stroke_nlvo[:-1], dp=3), 0.0)
pre_stroke_lvo = np.append(fudge_sum_one(pre_stroke_lvo[:-1], dp=3), 0.0)

In [65]:
mrs_dists['pre_stroke_nlvo'] = pre_stroke_nlvo
mrs_dists['pre_stroke_lvo'] = pre_stroke_lvo

In [66]:
print(mrs_dists['pre_stroke_nlvo'])
print(mrs_dists['pre_stroke_lvo'])

[0.583 0.163 0.104 0.101 0.042 0.007 0.   ]
[0.408 0.144 0.12  0.166 0.118 0.044 0.   ]


## LVO - no treatment

![](./images/data_sources_lvo-no-treatment.png)

In [67]:
mrs_dists['no_treatment_lvo'] = np.array([
    0.050, 0.079, 0.136, 0.164, 0.247, 0.135, 0.189])

In [68]:
np.sum(mrs_dists['no_treatment_lvo'])

1.0

## nLVO - no treatment

![](./images/data_sources_nlvo-no-treatment.png)

__1. Collect data__

In [69]:
no_treatment_nlvo_lvo = np.array(
    [0.1486, 0.2022, 0.1253, 0.1397, 0.1806, 0.0861, 0.1175])

In [70]:
no_treatment_nlvo_lvo = fudge_sum_one(no_treatment_nlvo_lvo, dp=3)

print(np.sum(no_treatment_nlvo_lvo), no_treatment_nlvo_lvo)

0.9999999999999999 [0.149 0.202 0.125 0.14  0.181 0.086 0.117]


__2. Find a reference probability__

In [71]:
p_mrsleq1_nlvo = 0.46

__3. Remove the LVO patients__

Assume that the combined nLVO and LVO mRS distribution is made up of a weighted sum of a separate nLVO and LVO distribution.

Define these probabilities for mRS <= 1: 
+ $P_1$ for nLVO & LVO,
+ $P_2$ for LVO,
+ $P_3$ for nLVO

And calculate the weight $w$.

The nLVO probability $P_3$ is found by:
1. scaling up the combined nLVO & LVO distribution to $(1 + w)$
1. scaling down the LVO distribution to $w$
2. taking the difference

As a formula, this is:

$$ P_3 = (1 + w) \times P_1 - w \times P_2$$

Rearrange to find the weight $w$:

$$ w \times (P_1 - P_2) = P_3 - P_1 $$

$$ w = \frac{P_3 - P_1}{P_1 - P_2} $$


In [72]:
p1 = np.sum(no_treatment_nlvo_lvo[:2])          # nLVO & LVO
p2 = np.sum(mrs_dists['no_treatment_lvo'][:2])  # LVO
p3 = p_mrsleq1_nlvo                             # nLVO

In [73]:
w = (p3 - p1) / (p1 - p2)

w

0.49099099099099125

Use this weight to calculate the new distribution:

In [74]:
no_treatment_nlvo = (1 + w) * no_treatment_nlvo_lvo - w * mrs_dists['no_treatment_lvo']

no_treatment_nlvo

array([0.19760811, 0.26239189, 0.1195991 , 0.12821622, 0.14859459,
       0.06194144, 0.08164865])

Round to the same precision as the no-treatment LVO distribution:

In [75]:
no_treatment_nlvo = fudge_sum_one(no_treatment_nlvo, dp=3)

print(np.round(np.sum(no_treatment_nlvo), 3), no_treatment_nlvo)

1.0 [0.198 0.262 0.12  0.128 0.148 0.062 0.082]


In [76]:
mrs_dists['no_treatment_nlvo'] = no_treatment_nlvo

Check that the mRS <= 1 values sum to the target probability:

In [77]:
np.sum(mrs_dists['no_treatment_nlvo'][:2])

0.46

Are the values the same to a few decimal places?

In [78]:
np.isclose(np.sum(mrs_dists['no_treatment_nlvo'][:2]), p_mrsleq1_nlvo)

True

## nLVO at no-effect time

![](./images/data_sources_nlvo-ivt-no-effect.png)

In [79]:
no_effect_nlvo_ivt_deaths = np.append(
    mrs_dists['no_treatment_nlvo'][:-1] * (1.0 - 0.011),
    mrs_dists['no_treatment_nlvo'][-1] + np.sum(mrs_dists['no_treatment_nlvo'][:-1] * 0.011),
)

In [80]:
no_effect_nlvo_ivt_deaths

array([0.195822, 0.259118, 0.11868 , 0.126592, 0.146372, 0.061318,
       0.092098])

In [81]:
no_effect_nlvo_ivt_deaths = fudge_sum_one(no_effect_nlvo_ivt_deaths, dp=3)

print(np.round(np.sum(no_effect_nlvo_ivt_deaths), 3), no_effect_nlvo_ivt_deaths)

1.0 [0.196 0.259 0.119 0.127 0.146 0.061 0.092]


In [82]:
mrs_dists['no_effect_nlvo_ivt_deaths'] = no_effect_nlvo_ivt_deaths

## LVO - IVT at no-effect time

![](./images/data_sources_lvo-ivt-no-effect.png)

In [83]:
no_effect_lvo_ivt_deaths = np.append(
    mrs_dists['no_treatment_lvo'][:-1] * (1.0 - 0.034),
    mrs_dists['no_treatment_lvo'][-1] + np.sum(mrs_dists['no_treatment_lvo'][:-1] * 0.034),
)

In [84]:
no_effect_lvo_ivt_deaths

array([0.0483  , 0.076314, 0.131376, 0.158424, 0.238602, 0.13041 ,
       0.216574])

In [85]:
no_effect_lvo_ivt_deaths = fudge_sum_one(no_effect_lvo_ivt_deaths, dp=3)

print(np.round(np.sum(no_effect_lvo_ivt_deaths), 3), no_effect_lvo_ivt_deaths)

1.0 [0.048 0.076 0.131 0.159 0.239 0.13  0.217]


In [86]:
mrs_dists['no_effect_lvo_ivt_deaths'] = no_effect_lvo_ivt_deaths

## LVO - MT at no-effect time

![](./images/data_sources_lvo-mt-no-effect.png)

In [87]:
no_effect_lvo_mt_deaths = np.append(
    mrs_dists['no_treatment_lvo'][:-1] * (1.0 - 0.040),
    mrs_dists['no_treatment_lvo'][-1] + np.sum(mrs_dists['no_treatment_lvo'][:-1] * 0.040),
)

In [88]:
no_effect_lvo_mt_deaths

array([0.048  , 0.07584, 0.13056, 0.15744, 0.23712, 0.1296 , 0.22144])

In [89]:
no_effect_lvo_mt_deaths = fudge_sum_one(no_effect_lvo_mt_deaths, dp=3)

print(np.round(np.sum(no_effect_lvo_mt_deaths), 3), no_effect_lvo_mt_deaths)

1.0 [0.048 0.076 0.131 0.157 0.237 0.13  0.221]


In [92]:
mrs_dists['no_effect_lvo_mt_deaths'] = no_effect_lvo_mt_deaths

## nLVO - IVT at time zero

![](data_sources_nlvo-ivt-time-zero.png)

__1. Get reference probability at no-effect time__

In [94]:
p_mrsleq1_tne = np.sum(mrs_dists['no_effect_nlvo_ivt_deaths'][:2])

p_mrsleq1_tne

0.455

__2. Calculate a reference probability at time zero.__

In [95]:
a = 0.76296

In [116]:
t = np.exp(a) * (p_mrsleq1_tne / (1.0 - p_mrsleq1_tne))
p_mrsleq1_t0 = round(t / (1 + t), 3)

p_mrsleq1_t0

0.642

__3. Combine mRS distributions.__

In [117]:
pre_stroke_nlvo_ivt_deaths = np.append(
    mrs_dists['pre_stroke_nlvo'][:-1] * (1.0 - 0.011),
    mrs_dists['pre_stroke_nlvo'][-1] + np.sum(mrs_dists['pre_stroke_nlvo'][:-1] * 0.011),
)

The time-zero distribution is a weighted combination of the pre-stroke and no-effect distributions.

Define these probabilities for mRS <= 1: 
+ $P_1$ for time-zero,
+ $P_2$ for pre-stroke,
+ $P_3$ for no-effect.

And calculate the weight $w$.

The time-zero probability $P_1$ is found by:
1. scaling down the no-effect distribution to $(1 - w)$
1. scaling down the pre-stroke distribution to $w$
2. taking the sum

As a formula, this is:

$$ P_1 = (1 - w) \times P_3 + w \times P_2$$

Rearrange to find the weight $w$:

$$ w \times (P_2 - P_3) = P_1 - P_3 $$

$$ w = \frac{P_1 - P_3}{P_2 - P_3} $$


In [118]:
p1 = p_mrsleq1_t0
p2 = np.sum(mrs_dists['pre_stroke_nlvo'][:2])
p3 = p_mrsleq1_tne

In [119]:
w = (p1 - p3) / (p2 - p3)

w

0.6426116838487973

Use this weight to calculate the new distribution:

In [120]:
time_zero_nlvo = (1 - w) * mrs_dists['no_effect_nlvo_ivt_deaths'] + w * mrs_dists['pre_stroke_nlvo']

time_zero_nlvo

array([0.44469072, 0.19730928, 0.10936082, 0.1102921 , 0.07916838,
       0.02629897, 0.03287973])

Round to the same precision as the no-treatment LVO distribution:

In [121]:
time_zero_nlvo = fudge_sum_one(time_zero_nlvo, dp=3)

print(np.round(np.sum(time_zero_nlvo), 3), time_zero_nlvo)

1.0 [0.445 0.197 0.11  0.11  0.079 0.026 0.033]


In [122]:
mrs_dists['t0_treatment_nlvo_ivt'] = time_zero_nlvo

Check that the mRS <= 1 values sum to the target probability:

In [123]:
np.sum(mrs_dists['t0_treatment_nlvo_ivt'][:2])

0.642

Are the values the same to a few decimal places?

In [125]:
np.isclose(np.sum(mrs_dists['t0_treatment_nlvo_ivt'][:2]), p_mrsleq1_t0)

True

## LVO - IVT at time zero

![](data_sources_lvo-ivt-time-zero.png)

__1. Get reference probability at no-effect time__

In [126]:
p_mrsleq1_tne = np.sum(mrs_dists['no_effect_lvo_ivt_deaths'][:2])

p_mrsleq1_tne

0.124

__2. Calculate a reference probability at time zero.__

In [127]:
a = 0.76296

In [128]:
t = np.exp(a) * (p_mrsleq1_tne / (1.0 - p_mrsleq1_tne))
p_mrsleq1_t0 = round(t / (1 + t), 3)

p_mrsleq1_t0

0.233

__3. Combine mRS distributions.__

In [129]:
pre_stroke_lvo_ivt_deaths = np.append(
    mrs_dists['pre_stroke_lvo'][:-1] * (1.0 - 0.034),
    mrs_dists['pre_stroke_lvo'][-1] + np.sum(mrs_dists['pre_stroke_lvo'][:-1] * 0.034),
)

The time-zero distribution is a weighted combination of the pre-stroke and no-effect distributions.

Define these probabilities for mRS <= 1: 
+ $P_1$ for time-zero,
+ $P_2$ for pre-stroke,
+ $P_3$ for no-effect.

And calculate the weight $w$.

The time-zero probability $P_1$ is found by:
1. scaling down the no-effect distribution to $(1 - w)$
1. scaling down the pre-stroke distribution to $w$
2. taking the sum

As a formula, this is:

$$ P_1 = (1 - w) \times P_3 + w \times P_2$$

Rearrange to find the weight $w$:

$$ w \times (P_2 - P_3) = P_1 - P_3 $$

$$ w = \frac{P_1 - P_3}{P_2 - P_3} $$


In [130]:
p1 = p_mrsleq1_t0
p2 = np.sum(mrs_dists['pre_stroke_lvo'][:2])
p3 = p_mrsleq1_tne

In [131]:
w = (p1 - p3) / (p2 - p3)

w

0.25467289719626174

Use this weight to calculate the new distribution:

In [132]:
time_zero_lvo = (1 - w) * mrs_dists['no_effect_lvo_ivt_deaths'] + w * mrs_dists['pre_stroke_lvo']

time_zero_lvo

array([0.13968224, 0.09331776, 0.1281986 , 0.16078271, 0.20818458,
       0.10809813, 0.16173598])

Round to the same precision as the no-treatment LVO distribution:

In [133]:
time_zero_lvo = fudge_sum_one(time_zero_lvo, dp=3)

print(np.round(np.sum(time_zero_lvo), 3), time_zero_lvo)

1.0 [0.14  0.093 0.128 0.161 0.208 0.108 0.162]


In [134]:
mrs_dists['t0_treatment_lvo_ivt'] = time_zero_lvo

Check that the mRS <= 1 values sum to the target probability:

In [135]:
np.sum(mrs_dists['t0_treatment_lvo_ivt'][:2])

0.233

Are the values the same to a few decimal places?

In [136]:
np.isclose(np.sum(mrs_dists['t0_treatment_lvo_ivt'][:2]), p_mrsleq1_t0)

True

## LVO - MT at time zero

![](./images/data_sources_lvo-mt-time-zero.png)

__2. Add excess deaths to pre-stroke distribution.__

In [137]:
pre_stroke_lvo_mt_deaths = np.append(
    mrs_dists['pre_stroke_lvo'][:-1] * (1.0 - 0.040),
    mrs_dists['pre_stroke_lvo'][-1] + np.sum(mrs_dists['pre_stroke_lvo'][:-1] * 0.040),
)

In [142]:
pre_stroke_lvo_mt_deaths = fudge_sum_one(pre_stroke_lvo_mt_deaths, dp=3)

print(np.round(np.sum(pre_stroke_lvo_mt_deaths), 3), pre_stroke_lvo_mt_deaths)

1.0 [0.392 0.138 0.115 0.16  0.113 0.042 0.04 ]


__3. Combine the data for full effect and no effect of recanalisation.__

In [144]:
time_zero_lvo_mt = (
    0.75 * pre_stroke_lvo_mt_deaths +
    0.25 * mrs_dists['no_effect_lvo_mt_deaths']
)

time_zero_lvo_mt

array([0.306  , 0.1225 , 0.119  , 0.15925, 0.144  , 0.064  , 0.08525])

In [145]:
time_zero_lvo_mt = fudge_sum_one(time_zero_lvo_mt, dp=3)

print(np.round(np.sum(time_zero_lvo_mt), 3), time_zero_lvo_mt)

1.0 [0.306 0.123 0.119 0.159 0.144 0.064 0.085]
